In [ ]:
import logging

import gym
import matplotlib.pyplot as plt
import numpy as np
import quanser_robots

import sys
#sys.load('../')
sys.path.insert(0,'../')
from Challenge_1.Algorithms.PolicyIteration import PolicyIteration
from Challenge_1.Algorithms.ValueIteration import ValueIteration
from Challenge_1.Models.NNModelPendulum import NNModelPendulum
from Challenge_1.Models.NNModelQube import NNModelQube
from Challenge_1.Models.SklearnModel import SklearnModel
from Challenge_1.util.ColorLogger import enable_color_logging
from Challenge_1.util.DataGenerator import DataGenerator
from Challenge_1.util.Discretizer import Discretizer
import itertools
from torch.optim.lr_scheduler import *
enable_color_logging(debug_lvl=logging.INFO)
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch
import torch.optim as optim

seed = 1234
# avoid auto removal of import with pycharm
quanser_robots

env_name = "Pendulum-v2"
#env_name = "Qube-v0"

## Settings

In [ ]:
n_samples = 10000
n_steps = 500 #10000
batch_size = 512
lr = 1e-3
path = "./NN-state_dict"

## Create the gym-environment

In [ ]:
env = gym.make(env_name)

## Create both neural net models

In [ ]:
if env_name == 'Pendulum-v2':
    dynamics_model = NNModelPendulum(n_inputs=env.observation_space.shape[0] + env.action_space.shape[0] + nb_angle_features,
                             n_outputs=env.observation_space.shape[0],
                             scaling=env.observation_space.high, optimizer='adam')

    reward_model = NNModelPendulum(n_inputs=env.observation_space.shape[0] + env.action_space.shape[0] + nb_angle_features,
                           n_outputs=1,
                           scaling=None, optimizer='adam')
elif env_name == 'Qube-v0':
    dynamics_model = NNModelQube(n_inputs=env.observation_space.shape[0] + env.action_space.shape[0] + nb_angle_features,
                         n_outputs=env.observation_space.shape[0],
                         scaling=env.observation_space.high, optimizer='adam')

    reward_model = NNModelQube(n_inputs=env.observation_space.shape[0] + env.action_space.shape[0] + nb_angle_features,
                           n_outputs=1,
                           scaling=None, optimizer='adam')

In [ ]:
lossfunction = nn.MSELoss()


## Create the training data

In [ ]:
state.shape

In [ ]:
def create_dataset(env_name, seed, n_samples):
    """
    """
    
    dg_train = DataGenerator(env_name=env_name, seed=seed)

    # s_prime - future state after you taken the action from state s
    state_prime, state, action, reward = dg_train.get_samples(n_samples)

    nb_angle_features = 1
    
    # replace the angle feature by 2 new features which are the sin(angle) and cos(angle)
    state_enhanced = np.zeros((len(state), state.shape[1]+nb_angle_features))
    state_enhanced[:, 0] = np.cos(state[:, 0])
    state_enhanced[:, 1] = np.sin(state[:, 0])
    state_enhanced[:, 2] = state[:, 1]

    # create training input pairs
    s_a_pairs = np.concatenate([state_enhanced, action[:, np.newaxis]], axis=1).reshape(-1, state_enhanced.shape[1] +
                                                                               env.action_space.shape[0])
    reward = reward.reshape(-1, 1)

    return s_a_pairs, state_prime, reward

In [ ]:
dg_train = DataGenerator(env_name=env_name, seed=seed)

# s_prime - future state after you taken the action from state s
state_prime, state, action, reward = dg_train.get_samples(n_samples)

In [ ]:
nb_angle_features = 1

In [ ]:
# replace the angle feature by 2 new features which are the sin(angle) and cos(angle)
state_enhanced = np.zeros((len(state), state.shape[1]+nb_angle_features))
state_enhanced[:, 0] = np.cos(state[:, 0])
state_enhanced[:, 1] = np.sin(state[:, 0])
state_enhanced[:, 2] = state[:, 1]

In [ ]:
# create training input pairs
s_a_pairs = np.concatenate([state_enhanced, action[:, np.newaxis]], axis=1).reshape(-1, state_enhanced.shape[1] +
                                                                           env.action_space.shape[0])
reward = reward.reshape(-1, 1)

In [ ]:
s_a_pairs.shape

In [ ]:
s_a_pairs_train, state_prime_train, reward_train = create_dataset(env_name, seed, n_samples)

### Create test input pairs

In [ ]:
dg_test = DataGenerator(env_name=env_name, seed=seed + 1)
s_prime, s, a, r = dg_test.get_samples(n_samples)
    
s_a = np.concatenate([s, a[:, np.newaxis]], axis=1).reshape(-1, env.observation_space.shape[0] +
                                                            env.action_space.shape[0])
r = r.reshape(-1, 1)
s_prime = s_prime.reshape(-1, env.observation_space.shape[0])

In [ ]:
s_a_pairs_test, state_prime_test, reward_test = create_dataset(env_name, seed+1, n_samples)

In [ ]:
X_low = np.concatenate([env.observation_space.low, env.action_space.low])
X_high = np.concatenate([env.observation_space.high, env.action_space.high])

In [ ]:
X_low = np.min(s_a_pairs_train, axis=0)
X_high = np.max(s_a_pairs_train, axis=0)

In [ ]:
def normalize_input(x):
    """
    Normalizes the input data by using linear scaling to [0,1]
    """
    x = x - X_low
    x /= (X_high - X_low)
    
    return x

## Normalize the input X for the neural network

In [ ]:
s_a_pairs_train = normalize_input(s_a_pairs_train)
s_a_pairs_test = normalize_input(s_a_pairs_test)

In [ ]:
X = torch.from_numpy(s_a_pairs).float()
Y = torch.from_numpy(state_prime).float()

X_val = torch.from_numpy(s_a)
Y_val = s_prime

#X -= 0.5
#Y = (Y - Y.min())
#Y /= Y.max()
#Y -= 0.5

model = dynamics_model
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4) #, weight_decay=1e12)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, lr, eta_min=0, last_epoch=-1)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
#scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

In [ ]:
plt.hist(X[:, 0])

In [ ]:
def validate_model(model, X, y):

    model.eval()

    with torch.no_grad():
        # y = torch.from_numpy(y).float()
        #X = torch.from_numpy(X).float()

        out = model(X)

        mse_test = ((out.detach().numpy() - y) ** 2).mean(axis=0)

        print("Test MSE: {}".format(mse_test))
    return mse_test.mean()

In [ ]:
def train(model, optimizer, X, Y, X_val, Y_val, n_epochs=150, batch_size=32):
    
    X = torch.from_numpy(X).float()
    Y = torch.from_numpy(Y).float()

    X_val = torch.from_numpy(X_val)
    Y_val = Y_val

    # https://stackoverflow.com/questions/45113245/how-to-get-mini-batches-in-pytorch-in-a-clean-and-efficient-way

    train_loss = []
    val_loss = []
    for epoch in range(n_epochs):

        # X is a torch Variable
        permutation = torch.randperm(X.size()[0])

        for i in range(0,X.size()[0], batch_size):
            optimizer.zero_grad()

            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X[indices], Y[indices]

            # in case you wanted a semi-full example
            outputs = model.forward(batch_x)
            loss = lossfunction(outputs,batch_y)

            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()

        if epoch % 25 == 0:
            for g in optimizer.param_groups:
                g['lr'] /= 2

        print("Step: {:d} -- total loss: {:3.8f}".format(epoch, loss.item()))
        val_loss.append(validate_model(model, X_val, Y_val))
        
    return train_loss, val_loss

## Start the training process

In [ ]:
train_loss_no_norm, val_loss_no_norm = train(dynamics_model, optimizer=optimizer,
                             X=s_a_pairs, Y=state_prime, X_val=s_a, Y_val=s_prime, n_epochs=50)

In [ ]:
train_loss, val_loss = train(dynamics_model, optimizer=optimizer,
                             X=s_a_pairs, Y=state_prime, X_val=s_a, Y_val=s_prime)

In [ ]:
train_loss_norm_sincos, val_loss_norm_sincos = train(dynamics_model, optimizer=optimizer,
                             X=s_a_pairs_train, Y=state_prime_train, X_val=s_a_pairs_test, Y_val=state_prime_test, n_epochs=50)

In [ ]:
plt.title('Qube-v0: Learning Dynamics\n Batch-Size=32, lr=1e-3, optimizer=Adam')
#plt.plot(train_loss, label='train_loss')
plt.plot(val_loss, label='val-loss with normalization')
plt.plot(val_loss_no_norm, label='val-loss no normalization')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()
plt.savefig('Qube_Dynamics_normalization_comparision.png')

In [ ]:
plt.title('Qube-v0: Learning Dynamics\n Batch-Size=32, lr=1e-3, optimizer=Adam')
#plt.plot(train_loss, label='train_loss')
plt.plot(val_loss, label='val-loss with normalization\nTest MSE: [0.009659 0.325742 0.617882 0.561754]')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()
plt.savefig('Qube_Dynamics_normalization.png', bbox='tight')

In [ ]:
train_loss, val_loss = train(dynamics_model, optimizer=optimizer,
                             X=s_a_pairs, Y=state_prime, X_val=s_a, Y_val=s_prime)

In [ ]:
torch.save(dynamics_model.state_dict(), "./NN-state_dict_dynamics")

In [ ]:
reward_model.train_network(s_a_pairs, reward, n_steps, path + "_reward")


In [ ]:
train_loss, val_loss = train(reward_model, optimizer=optimizer,
                             X=s_a_pairs, Y=reward, X_val=s_a, Y_val=r)

In [ ]:
torch.save(reward_model.state_dict(), "./NN-state_dict_reward")

In [ ]:
n_samples

In [ ]:
def start_policy_iteration(env_name, algorithm="vi", n_samples=10000, bins_state=50, bins_action=2, seed=1,
                           theta=1e-9, use_MC=True, MC_samples=500, dense_location=["center", "edge"]):
    env = gym.make(env_name)
    print("Training with {} samples.".format(n_samples))

    dg_train = DataGenerator(env_name=env_name, seed=seed)

    # s_prime - future state after you taken the action from state s
    state_prime, state, action, reward = dg_train.get_samples(n_samples)

    # create training input pairs
    s_a_pairs = np.concatenate([state, action[:, np.newaxis]], axis=1)

    # solve regression problem s_prime = f(s,a)
    # dynamics_model = SklearnModel(type="rf")
    # dynamics_model.fit(s_a_pairs, state_prime)
    #
    # # solve regression problem r = g(s,a)
    # reward_model = SklearnModel(type="rf")
    # reward_model.fit(s_a_pairs, reward)

    # But performance should not change much
    dynamics_model = NNModel(n_inputs=env.observation_space.shape[0] + env.action_space.shape[0],
                             n_outputs=env.observation_space.shape[0],
                             scaling=env.observation_space.high)

    reward_model = NNModel(n_inputs=env.observation_space.shape[0] + env.action_space.shape[0],
                           n_outputs=1,
                           scaling=None)

    dynamics_model.load_model("./NN-state_dict_dynamics") #_10000_200hidden")
    reward_model.load_model("./NN-state_dict_reward") #_10000_200hidden")

    # center, edge for pendulum is best for RF
    # edge, center is best for NN
    discretizer_state = Discretizer(n_bins=bins_state, space=env.observation_space,
                                    dense_locations=dense_location)
    discretizer_action = Discretizer(n_bins=bins_action, space=env.action_space)

    if algorithm == "pi":
        algo = PolicyIteration(env=env, dynamics_model=dynamics_model, reward_model=reward_model,
                               discretizer_state=discretizer_state, discretizer_action=discretizer_action, theta=theta)
    elif algorithm == "vi":
        algo = ValueIteration(env=env, dynamics_model=dynamics_model, reward_model=reward_model,
                              discretizer_state=discretizer_state, discretizer_action=discretizer_action, theta=theta,
                              use_MC=use_MC, MC_samples=MC_samples)
    else:
        raise NotImplementedError()

    algo.run(max_iter=100)

    return algo.policy, discretizer_action, discretizer_state

In [ ]:
policy, discretizer_action, discretizer_state = start_policy_iteration(env_name, seed=seed, n_samples=500)

In [ ]:
plt.plot(train_loss[60:])
plt.plot(val_loss[60:])

In [ ]:
plt.plot(train_loss[60:])
plt.plot(val_loss[60:])